In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [4]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [6]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
param_grid={'alpha':[0.001,0.01,1.0], 'force_alpha':[True,'bool'], 'fit_prior':[True,'bool'],'min_categories':[None,1,3]}
grid=GridSearchCV(CategoricalNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
135 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\naive_bayes.py", line 1378, in fit
    return super().fit(X, y, sample_weight=sample_weight)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\L

GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 1.0],
                         'fit_prior': [True, 'bool'],
                         'force_alpha': [True, 'bool'],
                         'min_categories': [None, 1, 3]},
             scoring='f1_weighted', verbose=3)

In [7]:
result=grid.cv_results_
grid_prediction=grid.predict(indep)

In [8]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,grid_prediction)
print(cm)

[[150   0]
 [  0 249]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(dep,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399



In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(dep,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'alpha': 0.001, 'fit_prior': True, 'force_alpha': True, 'min_categories': None} 1.0


In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(dep,grid.predict_proba(indep)[:,1])

1.0

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,param_min_categories,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023311,0.004644,0.014065,0.001916,0.001,True,True,None,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",0.975000,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.023960,0.003532,0.012250,0.004184,0.001,True,True,1,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",0.975000,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.019030,0.003812,0.011158,0.007153,0.001,True,True,3,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",0.975000,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.001208,0.001480,0.000000,0.000000,0.001,True,bool,None,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.003229,0.002573,0.000000,0.000000,0.001,True,bool,1,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,0.002063,0.004127,0.000000,0.000000,0.001,True,bool,3,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,0.000807,0.001614,0.000000,0.000000,0.001,bool,True,None,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,0.000245,0.000490,0.000000,0.000000,0.001,bool,True,1,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,0.001309,0.001293,0.000000,0.000000,0.001,bool,True,3,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,0.001135,0.001442,0.000000,0.000000,0.001,bool,bool,None,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
